In [1]:
import findspark


In [2]:
findspark.init('/Users/pranibansal/Downloads/spark-3.4.0-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/pranibansal/Downloads/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/pranibansal/.ivy2/cache
The jars for the packages stored in: /Users/pranibansal/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d0f97d55-b3c0-41b3-b3ad-d6bc761d5994;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in local-m2-cache
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in local-m2-cache
	found org.apache.commons#commons-pool2

In [16]:
import sys

from pyspark.sql import *
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.sql.functions import *
from pyspark.sql.types import *



In [53]:
df = spark \
    .read.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "first_topic") \
    .option("startingOffsets", "earliest") \
    .load() \
    

In [54]:
df.show()

23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+-------+--------------------+-----------+---------+------+--------------------+-------------+
|    key|               value|      topic|partition|offset|           timestamp|timestampType|
+-------+--------------------+-----------+---------+------+--------------------+-------------+
|   [30]|[7B 22 69 64 22 3...|first_topic|        0|     0|2023-06-04 15:53:...|            0|
|   [31]|[7B 22 69 64 22 3...|first_topic|        0|     1|2023-06-04 15:53:...|            0|
|   [32]|[7B 22 69 64 22 3...|first_topic|        0|     2|2023-06-04 15:53:...|            0|
|   [33]|[7B 22 69 64 22 3...|first_topic|        0|     3|2023-06-04 15:53:...|            0|
|   [34]|[7B 22 69 64 22 3...|first_topic|        0|     4|2023-06-04 15:53:...|            0|
|   [35]|[7B 22 69 64 22 3...|first_topic|        0|     5|2023-06-04 15:53:...|            0|
|   [36]|[7B 22 69 64 22 3...|first_topic|        0|     6|2023-06-04 15:53:...|            0|
|   [37]|[7B 22 69 64 22 3...|first_topic|        

In [58]:
# df = df.withColumn("json", from_json(df["json"], jsonSchema)) \
#                 .select("json.*")

# Parse value from binay to string
json_df = df.selectExpr("cast(value as string) as value")

# # Apply Schema to JSON value column and expand the value
# from pyspark.sql.functions import from_json

# json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], json_schema)).select("value.*") 

In [59]:
jsonSchema = StructType([StructField("id", StringType(), True), StructField("Year", StringType(), True),
                                     StructField("Reporting_Airline", StringType(), True)])

In [60]:

json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], jsonSchema)).select("value.*") 

In [61]:
json_expanded_df.show()

23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---+----+-----------------+
| id|Year|Reporting_Airline|
+---+----+-----------------+
|  0|1998|               NW|
|  1|2009|               FL|
|  2|2013|               MQ|
|  3|2010|               DL|
|  4|2006|               US|
|  5|1995|               DL|
|  6|2006|               CO|
|  7|2019|               9E|
|  8|2008|               YV|
|  9|2018|               WN|
| 10|1991|               US|
| 11|2014|               WN|
| 12|1994|               AA|
| 13|2013|               OO|
| 14|2003|               UA|
| 15|1988|               PI|
| 16|2007|               NW|
| 17|2015|               AS|
| 18|2006|               UA|
| 19|2017|               WN|
+---+----+-----------------+
only showing top 20 rows



In [62]:
json_expanded_df.select('Reporting_Airline').groupBy('Reporting_Airline').count().show()

23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+-----------------+-----+
|Reporting_Airline|count|
+-----------------+-----+
|               US|   40|
|               YV|    6|
|               WN|   72|
|               PI|    4|
|               EV|   11|
|               HP|   11|
|               NW|   25|
|               MQ|   22|
|               DL|   77|
|               OO|   27|
|               UA|   39|
|               XE|    8|
|               EA|    1|
|               YX|    2|
|               TW|    9|
|               B6|    7|
|               F9|    4|
|               PS|    1|
|               FL|    5|
|               CO|   31|
+-----------------+-----+
only showing top 20 rows



In [65]:
df_filtered = json_expanded_df.select('id','Year','Reporting_Airline').filter('Year >= 2015')